In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Listing current data on our folder.
import os
print(os.listdir("."))

['CSVs', 'FIGs', 'Midi notes extraction.ipynb', 'MIDIs', 'Referensi.txt']


In [6]:
# Defining some constants and creating a new folder for MIDIs.
midi_path = "MIDIs"

# Some helper methods.    
def concat_path(path, child):
    return path + "/" + child

# Look for all midis in midi_path
midis_filename = os.listdir(midi_path)
print(midis_filename)

['00024.mid', '00025.mid', '00065.mid', '00075.mid', '00080.mid', '00083.mid', '00089.mid', '00101.mid', '00106.mid', '00108.mid', '00110.mid', '00111.mid', '00114.mid', '00116.mid', '00120.mid', '00128.mid', '00129.mid', '00133.mid', '00139.mid', '00140.mid', '00160.mid', '00162.mid', '00167.mid', '00185.mid', '00192.mid', '00193.mid', '00195.mid', '00197.mid', '00210.mid', '00232.mid', '00233.mid', '00245.mid', '00250.mid', '00265.mid', '00271.mid', '00276.mid', '00278.mid', '00279.mid', '00289.mid', '00293.mid', '00306.mid', '00319.mid', '00321.mid', '00362.mid', '00363.mid', '00392.mid', '00399.mid', '00438.mid', '00461.mid', '00468.mid', '00475.mid', '00513.mid', '00514.mid', '00515.mid', '00529.mid', '00533.mid', '00540.mid', '00550.mid', '00561.mid', '00562.mid', '00568.mid', '00570.mid', '00572.mid', '00573.mid', '00594.mid', '00606.mid', '00628.mid', '00636.mid', '00643.mid', '00647.mid', '00660.mid', '00662.mid', '00685.mid', '00693.mid', '00728.mid', '00730.mid', '00732.mid'

In [7]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch

def open_midi(midi_path):
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()       

    return midi.translate.midiFileToStream(mf)

base_midis = []

# Open and append all midis in midi_path in to base_midi
for midis in midis_filename:
    base_midis.append(open_midi(concat_path(midi_path, midis)))

In [8]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

def save_to_csv(x,y,n):
    df = pd.DataFrame({'beats': x, 'note_index': y})
    df.to_csv("CSVs/"+midis_filename[n]+".csv", index=False)

def extract_notes(midi_part):
    parent_element = []
    ret = []
    for nt in midi_part.flat.notes:        
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            for pitch in nt.pitches:
                ret.append(max(0.0, pitch.ps))
                parent_element.append(nt)

    return ret, parent_element

def print_parts_countour(midi,n):
    fig = plt.figure(figsize=(12, 5))
    ax = fig.add_subplot(1, 1, 1)
    minPitch = pitch.Pitch('C10').ps
    maxPitch = 0
    xMax = 0
    
    # Drawing notes.
    for i in range(len(midi.parts)):
        top = midi.parts[i].flat.notes                  
        y, parent_element = extract_notes(top)
        if (len(y) < 1): continue
            
        x = [n.offset for n in parent_element]
        ax.scatter(x, y, alpha=0.6, s=7)

        # save notes to csv
        save_to_csv(x,y,n)
        
        aux = min(y)
        if (aux < minPitch): minPitch = aux
            
        aux = max(y)
        if (aux > maxPitch): maxPitch = aux
            
        aux = max(x)
        if (aux > xMax): xMax = aux
    
    for i in range(1, 10):
        linePitch = pitch.Pitch('C{0}'.format(i)).ps
        if (linePitch > minPitch and linePitch < maxPitch):
            ax.add_line(mlines.Line2D([0, xMax], [linePitch, linePitch], color='red', alpha=0.1))        

    plt.ylabel("Note index (each octave has 12 notes)")
    plt.xlabel("Number of quarter notes (beats)")
    plt.title('Voices motion approximation, each color is a different instrument, red lines show each octave')
    
    # save plot to figure
    plt.savefig("FIGs/"+midis_filename[n]+".png")
    # plt.show()
    plt.close()

# Focusing only on 6 first measures to make it easier to understand.
# print_parts_countour(base_midi.measures(0, 6))

for n in range(len(base_midis)):
    print_parts_countour(base_midis[n],n)